In [1]:
# for article content scripting with BeautifulSoup
import requests
from bs4 import BeautifulSoup
import re

# for rendering html template with Jinja
from jinja2 import Environment, FileSystemLoader
import os

# for building and sending emails with MailChimp
import mailchimp_marketing as MailchimpMarketing
from mailchimp_marketing.api_client import ApiClientError
import json

# for viewing the email draft in webpage
import webbrowser
from datetime import date

import config as cf
import importlib
cf = importlib.reload(cf)

In [114]:
video_url_list = ['https://www.epochtimes.com/b5/21/4/15/n12882404.htm',
                  'https://www.epochtimes.com/b5/21/4/15/n12882514.htm',
                  'https://www.epochtimes.com/b5/21/4/15/n12882105.htm',
                  'https://www.epochtimes.com/b5/21/4/12/n12874815.htm']

video_page_url = 'https://www.epochtimes.com/gb/ncid1247351.htm'

In [115]:
page = requests.get(video_page_url.replace('/gb/', '/b5/'))

In [116]:
video_page_soup = BeautifulSoup(page.text, 'html.parser')

In [119]:
img_url = video_page_soup.select_one(f"a[href='{video_url_list[1].replace('/gb/', '/b5/')}']").find('img').get('src')

In [120]:
img_url

'https://i.epochtimes.com/assets/uploads/2021/04/id12871696-731c1292faf8ecb1db04b660aa2871c3-600x400.jpg'

# Define the Article Object with BeautifulSoup

In [2]:
class Article():
    
    def __init__(self, url, title=None, img_url=None, preview_text=None, tag=None, label=None, class_name='article', id_name='main', video_page_url=None):
        
        self._url = url.replace('/gb/', '/b5/')
        self._title = title
        self._img_url = img_url
        self._preview_text = preview_text
        self._tag = tag
        self._label = label
        self._video_page_url = video_page_url.replace('/gb/', '/b5/')
        
        page = requests.get(self._url)
        self._soup = BeautifulSoup(page.text, 'html.parser')
        
        self._article = self._soup.find(class_=class_name)
        
        if not self._article:
            self._article = self._soup.find(id=id_name)
            
        if not self._article:
            self._article = self._soup.find('article')
        
    @property
    def url(self):
        return self._url
    
    @url.setter
    def url(self, value):
        self._url = value
        
    @property
    def title(self, class_name='title'):
        
        if not self._title:
            article_title = self._article.find(class_=class_name)
            self._title = article_title.text
            
        return self._title
    
    @title.setter
    def title(self, value):
        self._title = value
    
    @property
    def img_url(self, class_name='featured_image'):
        
        if not self._img_url:
            try:
                self._img_url = self._article.find(class_=class_name).find('a').get('href')
            except:
                try:
                    self._img_url = self._article.find(class_=class_name).find('figure').find('img').get('src')
                except:
                    try:
                        self._img_url = self._article.find(class_='wp-post-image').get('src')
                    except:
                        page = requests.get(self._video_page_url)
                        video_page_soup = BeautifulSoup(page.text, 'html.parser')
                        self._img_url = video_page_soup.select_one(f"a[href='{self._url}']").find('img').get('src')
        
        return self._img_url
    
    @img_url.setter
    def img_url(self, value):
        self._img_url = value
    
    @property
    def preview_text(self, id_name='artbody'):
        
        paragraph_index = 0
        
        while not self._preview_text:
            article_first_paragraph = self._article.find(id=id_name).find_all('p')[paragraph_index].text
            article_preview_text = re.sub(r'\【[^)]*\】', '', article_first_paragraph)
            article_preview_text = re.sub(r'\n', '', article_preview_text) 
            article_preview_text = re.sub(r'\（英文大纪元[^)]*\编译）', '', article_preview_text)
            article_preview_text = re.sub(r'\（英文大紀元[^)]*\編譯）', '', article_preview_text)
            article_preview_text = re.sub(r'\（大紀元[^)]*\報導）', '', article_preview_text)
            article_preview_text = re.sub(r'\（大紀元[^)]*\報導）', '', article_preview_text)
            self._preview_text = re.sub(r'\（大纪元[^)]*\报导）', '', article_preview_text)
            paragraph_index += 1
        
        return self._preview_text
    
    @preview_text.setter
    def preview_text(self, value):
        self._preview_text = value
        
    @property
    def tag(self, id_name='breadcrumb'):
        
        if not self._tag:
            breadcrumb = self._soup.find(id=id_name)
            raw_tag = breadcrumb.findAll('a')[-1].text
            raw_tag = re.sub(r'\([^)]*\)', '', raw_tag)
            self._tag = re.sub(r'\（[^)]*\）', '', raw_tag)
        
        return self._tag
    
    @tag.setter
    def tag(self, value):
        self._tag = value
        
    @property
    def label(self):
        return self._label
    
    @label.setter
    def label(self, value):
        self._label = value

In [3]:
class Content(Article):
    
    def __init__(self, article_url_list, video_url_list, video_page_url=None):
        
        # parse data
        self.article_list = [Article(article_url, video_page_url=video_page_url) for article_url in article_url_list]
        
        self.video_list = [Article(video_url, video_page_url=video_page_url) for video_url in video_url_list]
        self.video_pair_list = [[value, self.video_list[counter+1]] for counter, value in enumerate(self.video_list) if counter%2 == 0]      

In [4]:
class Jinja_Template(Content):
    
    def __init__(self, 
                 template_folder_name = 'Jinja Templates', 
                 template_file_name = 'newsletter_as_jinja_template_base.html'):

        file_loader = FileSystemLoader('../'+template_folder_name+'/')
        self.env = Environment(loader=file_loader, extensions=['jinja2.ext.do'])

        # render the base template
        self.template = self.env.get_template(template_file_name)
        
    def render(self, source_contect, 
               folder_name='Exported HTML', 
               rendered_newsletter_filename = 'rendered_newsletter.html'):
        
        # description: subject_line and preview_text
        self.subject_line = source_contect.article_list[0].title
        article_title_list = [article.title for article in source_contect.article_list]
        self.preview_text = ' · '.join(article_title_list[1:])
        
        self.rendered_content = self.template.render(env=self.env, 
                                                   header=source_contect.article_list[0], 
                                                   article_list=source_contect.article_list, 
                                                   video_pair_list=source_contect.video_pair_list)
        
        self.folder_path = '../' + folder_name + '/'
        
        if not os.path.exists(self.folder_path):
            os.makedirs(self.folder_path)
        
        self.newsletter_path = self.folder_path + rendered_newsletter_filename
        
        with open(self.newsletter_path, "w") as fh:
            fh.write(self.rendered_content)
            
        print('newsletter rendered at: '+ self.newsletter_path)
        
    def preview(self):

        webbrowser.open('file://' + os.path.realpath(self.newsletter_path))

In [5]:
class MailChimp_Campaign(Jinja_Template):

    def __init__(self, newsletter, mailchimp_config,
                 # sender information
                 from_name='Anonymous Sender',
                 reply_to='no_reply@email.com'):

        self.subject_line = newsletter.subject_line
        self.preview_text = newsletter.preview_text
        self.rendered_content = newsletter.rendered_content
        self.mailchimp_config = mailchimp_config

        # sender information
        self.from_name = from_name
        self.reply_to = reply_to
        
        # campaign name
        today = date.today().strftime("%d %B %Y")
        self.campaign_title = 'Campaign_Created_through_API' + today

    def connect_to_MailChimp(self):

        self.client = MailchimpMarketing.Client()
        self.client.set_config({
            "api_key": self.mailchimp_config["API_KEY"],
            "server": self.mailchimp_config["API_KEY"].split('-')[-1]
        })

        response_ping = self.client.ping.get()

        if response_ping['health_status'] == "Everything's Chimpy!":
            print("MailChimp account connected!")

    def update_MailChimp_template(self):

        response = self.client.templates.update_template(
            self.mailchimp_config['template_id'], {"name": "Template_Created_through_API", "html": self.rendered_content})
        print("Template updated at:", response['date_edited'])

    def specify_recipients(self):

        if self.mailchimp_config['segment_id']:
            self.recipients = {'segment_opts':
                               {'saved_segment_id':
                                   self.mailchimp_config['segment_id']},
                               'list_id': self.mailchimp_config['list_id']}
        else:
            self.recipients = {'list_id': self.mailchimp_config['list_id']}

        print('recipients specified.')

    def create(self):

        self.connect_to_MailChimp()
        self.update_MailChimp_template()
        self.specify_recipients()

        self.created_campaign = self.client.campaigns.create({"type": "regular",
                                                              'recipients': self.recipients,
                                                              "settings":
                                                              {"title": self.campaign_title,
                                                               "subject_line": self.subject_line,
                                                               "preview_text": self.preview_text,
                                                               "from_name": self.from_name,
                                                               "reply_to": self.reply_to,
                                                               "template_id": self.mailchimp_config['template_id'],
                                                               "auto_footer": False}})
        print('Campaign created successfully!')

    def send(self):

        campaign_id = self.created_campaign['id']
        response_of_campaigns_send = self.client.campaigns.send(campaign_id)

        return response_of_campaigns_send

# Processing Part

In [6]:
article_url_list = ['https://www.epochtimes.com/b5/21/4/23/n12900319.htm',
                    'https://www.epochtimes.com/b5/21/4/22/n12898316.htm',
                    'https://www.epochtimes.com/b5/21/4/22/n12898029.htm',
                    'https://www.epochtimes.com/b5/21/4/22/n12898308.htm']

video_url_list = ['https://www.epochtimes.com/b5/21/4/24/n12902785.htm',
                  'https://www.epochtimes.com/b5/21/4/24/n12902456.htm',
                  'https://www.ntdtv.com/b5/2021/04/24/a103103154.html',
                  'https://www.ntdtv.com/b5/2021/04/25/a103104026.html']

video_page_url = 'https://www.epochtimes.com/gb/ncid1247351.htm'

In [7]:
source_content = Content(article_url_list, video_url_list, video_page_url=video_page_url)

In [8]:
for article in source_content.article_list:
    print(article.title)
    print(article.tag)
    print(article.label)
    print(article.preview_text)
    print()

組圖：維吾爾族英國集會控訴中共犯種族滅絕罪
國際要聞
None
4月22日，維吾爾族成員於英國倫敦舉行集會抗議，呼籲英國議會承認中共對維吾爾族穆斯林犯下種族滅絕罪和反人類罪。

英國最貴豪宅 中國富豪重新裝修恐需2億鎊
社會新聞
None
英國媒體披露，去年一位來自重慶、在香港發家的中國富豪購買了倫敦騎士橋的一棟豪宅，成交價達到2.1億鎊，是當時英國最貴的豪宅，現在這位富豪要重新裝修房子，可能還需要再拿出2億鎊。

組圖：英超第32輪 曼城2:1勝阿斯頓維拉
國際足球
None
4月21日，英國伯明翰，2020/21賽季英超第32輪比賽，曼徹斯特城隊（簡稱「曼城」）與阿斯頓維拉隊的比賽在維拉公園球場（Villa Park）舉行，曼城以2:1勝阿斯頓維拉。

英神經毒劑案 兩俄特工與捷克爆炸案有關
社會新聞
None
捷克共和國警方表示，涉嫌於2018年在英國使用神經毒劑投毒的兩名俄國人，跟該國2014年發生的導致兩人死亡的爆炸案有關。俄羅斯特工還涉嫌向一名軍火商投毒。



In [110]:
# source_content.article_list[0].title = '疫情导致去年英国经济缩水创300年记录'

In [9]:
for article in source_content.video_list:
    try:
        print(article.title)
        print(article.tag)
        print(article.preview_text)
        print(article.img_url)
        print()
    except:
        pass

紀念四‧二五詩朗誦：《永恆的希望》
每日新聞
你的臉龐平靜祥和如同晨煉場上初升的暖陽可我知道你站在那裡紅牆之外懷著擔當
https://i.epochtimes.com/assets/uploads/2020/10/0495326fe7e58dcf8ff9471e7a2a529e-320x200.jpg

【拍案驚奇】日防長警告防台赤化 日媒：別再救中共
新聞拍案驚奇
大家好，歡迎收看《新聞拍案驚奇》，我是大宇。
https://i.epochtimes.com/assets/uploads/2021/04/id12902492-4eb44cc4941a25c091e950adfc506d55-320x200.jpg

【熱點互動】專訪劉慧卿：香港黑暗的日子
熱點互動直播


In [10]:
source_content.video_list[1].img_url

'https://i.epochtimes.com/assets/uploads/2021/04/id12902492-4eb44cc4941a25c091e950adfc506d55-320x200.jpg'

In [12]:
for video in source_content.video_list:
    print(video.img_url)

https://i.epochtimes.com/assets/uploads/2020/10/0495326fe7e58dcf8ff9471e7a2a529e-320x200.jpg
https://i.epochtimes.com/assets/uploads/2021/04/id12902492-4eb44cc4941a25c091e950adfc506d55-320x200.jpg
https://i.ntdtv.com/assets/uploads/2021/04/maxresdefault-81-800x450.jpg
https://i.ntdtv.com/assets/uploads/2021/04/thumbnail_d-1-7-800x450.jpg


In [29]:
source_content.video_list[0].img_url = 'https://i.ntdtv.com/assets/uploads/2021/04/maxresdefault-2-16-800x450.jpg'

In [36]:
source_content.video_list[1].img_url = 'https://i.epochtimes.com/assets/uploads/2021/04/id12871696-731c1292faf8ecb1db04b660aa2871c3-600x400.jpg'

In [37]:
source_content.video_list[2].img_url = 'https://i.epochtimes.com/assets/uploads/2021/04/id12871426-a948bfecbdb0af2e8292697829ecb2a5-320x200.jpg'

In [38]:
source_content.video_list[3].img_url = 'https://i.epochtimes.com/assets/uploads/2021/04/id12871886-51960545c5e2f1bfd1522d9ccab9f3d5-320x200.jpg'

In [13]:
newsletter = Jinja_Template()

In [14]:
newsletter.render(source_content)

newsletter rendered at: ../Exported HTML/rendered_newsletter.html


In [15]:
newsletter.preview()

# Connect to MailChimp

## Test Campaign

In [16]:
mailchimp_config_test = {
    "API_KEY": cf.mailchimp_config_test.API_KEY,
    "template_id": cf.mailchimp_config_test.template_id,
    "segment_id": cf.mailchimp_config_test.segment_id, # recipients
    "list_id": cf.mailchimp_config_test.list_id # members list
}

### Create MailChimp campaign

In [17]:
test_campaign = MailChimp_Campaign(newsletter, mailchimp_config_test, 
                                   from_name=cf.mailchimp_compaign.from_name, 
                                   reply_to=cf.mailchimp_compaign.reply_to)

In [18]:
test_campaign.create()

MailChimp account connected!
Template updated at: 2021-04-25T21:47:59+00:00
recipients specified.
Campaign created successfully!


In [19]:
test_campaign.send()

<Response [204]>

## Real Campaign

In [20]:
mailchimp_config_real = {
    "API_KEY": cf.mailchimp_config_real.API_KEY,
    "template_id": cf.mailchimp_config_real.template_id,
    "segment_id": cf.mailchimp_config_real.segment_id, # recipients
    "list_id": cf.mailchimp_config_real.list_id # members list
}

In [21]:
real_campaign = MailChimp_Campaign(newsletter, mailchimp_config_real, 
                                   from_name=cf.mailchimp_compaign.from_name, 
                                   reply_to=cf.mailchimp_compaign.reply_to)

In [22]:
real_campaign.create()

MailChimp account connected!
Template updated at: 2021-04-25T21:48:15+00:00
recipients specified.
Campaign created successfully!


In [23]:
real_campaign.send()

<Response [204]>

## Drafts

In [200]:
# define article tag
tag_list = ['英国新闻', '英国新闻', '英国新闻', '英国新闻']

for index in range(len(tag_list)):
    article_list[index].tag = tag_list[index]
    
# article_list[0].img_url = 'https://i.epochtimes.com/assets/uploads/2020/12/2012112056442124.jpg'